# ENSEMBLE Learning


# Q1

If you have trained five different models on the exact same training data,
and they all achieve 95% precision, is there any chance that you can
combine these models to get better results? If so, how? If not, why?


**_Yes by using ensemble learning methods you can try to experiment , combine these models and increase score to get even better results_**


# Q2

What is the difference between hard and soft voting classifiers?

**_Hard voting clfs typically use the predictions made by the models however in soft voting the mean off probs are taken to predict the final prediction_**


# Q3

Is it possible to speed up training of a bagging ensemble by distributing
it across multiple servers? What about pasting ensembles, boosting
ensembles, random forests, or stacking ensembles?

**_The thing to note is that the workflow of bagging pasting and randomforest is parallel where a subset of training dataset is given to models to train over and each model is INDEPENDANT... however the boosting and stacking ensembles have a SEQUENTIAL workflow where during the training iterations the next model is always depended upon the current one like in boosting new moedel is trained over the predictions of prev (in some way) and in stacking the individual models can be trained across multiple server but the model/s in the next layer is dependant on the previous(hmmm not too sure rn)_**


# Q4

What is the benefit of out-of-bag evaluation

**_Some instances are left out from the training set during the training of Bagging ensemble but these can be utilised to eval the model without needing the seperate valid set_**


# Q5

What makes extra-trees ensembles more random than regular random
forests? How can this extra randomness help? Are extra-trees classifiers
slower or faster than regular random forests?

**_by tweaking the hyperparam "splitter" we can make extra tree.... these trees don't try to find the best feature to make a split over but they choose best feature from a randomly selected subset... this extra randomness can be utilised to help reduce the overfititing problem (but the trade-off is still there)... extra trees are definitly faster as they remove the most time consuming step that is "Looking for the Best Feature_**


# Q6

If your AdaBoost ensemble underfits the training data, which
hyperparameters should you tweak, and how?

**_Can try increasing number of estimators and reduce regularisation hyperparam of base estimator... Also inc the Learing Rate_**


# Q7

If your gradient boosting ensemble overfits the training set, should you
increase or decrease the learning rate?

**_to stop overfitting we should dec the lr and inc the number of estimators [This is called SHRINKAGE]_**


# Q8

Load the MNIST dataset (introduced in Chapter 3), and split it into a
training set, a validation set, and a test set (e.g., use 50,000 instances for
training, 10,000 for validation, and 10,000 for testing). Then train
various classifiers, such as a random forest classifier, an extra-trees
classifier, and an SVM classifier. Next, try to combine them into an
ensemble that outperforms each individual classifier on the validation
set, using soft or hard voting. Once you have found one, try it on the test
set. How much better does it perform compared to the individual
classifiers?


In [114]:
# already have it doanloaded on my system so make sure u download it from kaggle to and load both train and test csvs
# this cell is  for the libraries only
import os
import sys
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import (
    RandomForestClassifier,
    ExtraTreesClassifier,
    VotingClassifier,
    StackingClassifier,
)
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [5]:
data_path = r"C:\Users\whate\Downloads\DATASETS\MNIST"

train = pd.read_csv(os.path.join(data_path, "mnist_train.csv"))
test = pd.read_csv(os.path.join(data_path, "mnist_test.csv"))

# Splitting


In [6]:
train[:5]

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
y_train = train["label"]
X_train = train.drop(columns="label")

In [11]:
y_test = test["label"]
X_test = test.drop(columns="label")

In [16]:
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)

In [33]:
rf_clf = RandomForestClassifier(random_state=42, n_estimators=100)
ext_clf = ExtraTreesClassifier(random_state=42)
log_clf = LogisticRegression(random_state=42, penalty="l2", max_iter=1000)
svm_clf = LinearSVC(random_state=42, dual=True, max_iter=100, tol=20)

In [34]:
estimators = [rf_clf, ext_clf, log_clf, svm_clf]
for estimator in estimators:
    estimator.fit(X_tr, y_tr)

c:\Users\whate\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [35]:
[estimator.score(X_val, y_val) for estimator in estimators]

[0.96925, 0.97225, 0.915, 0.82275]

# Voting Classifier


In [37]:
named_estimator = [
    ("rf_clf", rf_clf),
    ("ext_clf", ext_clf),
    ("log_clf", log_clf),
    ("svm_clf", svm_clf),
]

In [38]:
voting_clf = VotingClassifier(named_estimator)

In [39]:
voting_clf.fit(X_tr, y_tr)

c:\Users\whate\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


VotingClassifier(estimators=[('rf_clf',
                              RandomForestClassifier(random_state=42)),
                             ('ext_clf', ExtraTreesClassifier(random_state=42)),
                             ('log_clf',
                              LogisticRegression(max_iter=1000,
                                                 random_state=42)),
                             ('svm_clf',
                              LinearSVC(dual=True, max_iter=100,
                                        random_state=42, tol=20))])

In [40]:
voting_clf.score(X_val, y_val)

0.9533333333333334

In [41]:
# now evaluating the clones
y_val_encoded = y_val.astype(np.int64)

In [42]:
[estimator.score(X_val, y_val_encoded) for estimator in estimators]

[0.96925, 0.97225, 0.915, 0.82275]

In [43]:
# removing SVM to check if score incs

voting_clf.set_params(svm_clf="drop")

VotingClassifier(estimators=[('rf_clf',
                              RandomForestClassifier(random_state=42)),
                             ('ext_clf', ExtraTreesClassifier(random_state=42)),
                             ('log_clf',
                              LogisticRegression(max_iter=1000,
                                                 random_state=42)),
                             ('svm_clf', 'drop')])

In [44]:
# didn't drop from trained estimator clones so to do that we have to remove from both estimators_ and named_estimators_
voting_clf.estimators_

[RandomForestClassifier(random_state=42),
 ExtraTreesClassifier(random_state=42),
 LogisticRegression(max_iter=1000, random_state=42),
 LinearSVC(dual=True, max_iter=100, random_state=42, tol=20)]

In [45]:
svm_clf_trained = voting_clf.named_estimators_.pop("svm_clf")
voting_clf.estimators_.remove(svm_clf_trained)

In [46]:
# eval again
voting_clf.score(X_val, y_val)  # lmao svm was noob holding us back

0.97

In [47]:
# trying soft
voting_clf.voting = "soft"
voting_clf.score(X_val, y_val)

# nah hard voting wins

0.9515833333333333

In [48]:
# trying on test
voting_clf.voting = "hard"
voting_clf.score(X_test, y_test)
# noice

0.969

In [49]:
[estimator.score(X_test, y_test) for estimator in estimators]

[0.9686, 0.9702, 0.914, 0.8218]

In [50]:
[
    estimator.score(X_test, y_test.astype(np.int64))
    for estimator in voting_clf.estimators_
]

# extra trees is slayiong

[0.9686, 0.9702, 0.914]

In [58]:
voting_clf.set_params(log_clf="drop")

VotingClassifier(estimators=[('rf_clf',
                              RandomForestClassifier(random_state=42)),
                             ('ext_clf', ExtraTreesClassifier(random_state=42)),
                             ('log_clf', 'drop'), ('svm_clf', 'drop')])

In [60]:
# trying to remove this log_clf too
log_clf_train = voting_clf.named_estimators_.pop("log_clf")
voting_clf.estimators_.remove(log_clf_train)

In [61]:
voting_clf.estimators_

[RandomForestClassifier(random_state=42),
 ExtraTreesClassifier(random_state=42)]

In [62]:
voting_clf.score(X_val, y_val)

0.9701666666666666

In [63]:
voting_clf.score(X_test, y_test)

0.9695

In [65]:
voting_clf.voting = "soft"
voting_clf.score(X_test, y_test)
# lmao soft voting w

0.9707

# Q9

`Run the individual classifiers from the previous exercise to make
predictions on the validation set, and create a new training set with the resulting predictions: each training instance is a vector containing the set
of predictions from all your classifiers for an image, and the target is the
image’s class. Train a classifier on this new training set. Congratulations
—you have just trained a blender, and together with the classifiers it
forms a stacking ensemble! Now evaluate the ensemble on the test set.
For each image in the test set, make predictions with all your classifiers,
then feed the predictions to the blender to get the ensemble’s
predictions. How does it compare to the voting classifier you trained
earlier? Now try again using a StackingClassifier instead. Do you get
better performance? If so, why?`


In [97]:
# looking at book no idea how to do this lol
X_val_preds = np.empty((len(X_val), 2), dtype=object)

In [98]:
X_val_preds.shape  # so shape ia equal to

(12000, 2)

In [99]:
estimators[:2]

[RandomForestClassifier(random_state=42),
 ExtraTreesClassifier(random_state=42)]

In [100]:
for idx, estimator in enumerate(estimators[:2]):
    X_val_preds[:, idx] = estimator.predict(X_val)

In [101]:
X_val_preds

array([[7, 7],
       [3, 3],
       [8, 8],
       ...,
       [9, 9],
       [7, 7],
       [2, 2]], dtype=object)

In [102]:
rf_blend = RandomForestClassifier(n_estimators=200, oob_score=True, random_state=42)
rf_blend.fit(X_val_preds, y_val)

RandomForestClassifier(n_estimators=200, oob_score=True, random_state=42)

In [103]:
rf_blend.oob_score_

0.9699166666666666

In [104]:
X_test_preds = np.empty((len(X_test), len(estimator[:2])), dtype=object)

In [106]:
for idx, estimator in enumerate(estimators[:2]):
    X_test_preds[:, idx] = estimator.predict(X_test)

In [107]:
y_preds = rf_blend.predict(X_test_preds)

In [109]:
accuracy_score(y_test, y_preds)

0.969

# Stacking

`Since StackingClassifier uses K-Fold cross-validation, we don't need a separate validation set, so let's join the training set and the validation set into a bigger training set:`


In [133]:
X_full, y_full = train.drop(columns="label"), train["label"]

In [136]:
stack_clf = StackingClassifier(named_estimator, final_estimator=rf_blend)
stack_clf.fit(X_full, y_full)

c:\Users\whate\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\whate\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/st

StackingClassifier(estimators=[('rf_clf',
                                RandomForestClassifier(random_state=42)),
                               ('ext_clf',
                                ExtraTreesClassifier(random_state=42)),
                               ('log_clf',
                                LogisticRegression(max_iter=1000,
                                                   random_state=42)),
                               ('svm_clf',
                                LinearSVC(dual=True, max_iter=100,
                                          random_state=42, tol=20))],
                   final_estimator=RandomForestClassifier(n_estimators=200,
                                                          oob_score=True,
                                                          random_state=42))

In [137]:
stack_clf.score(X_test, y_test)

1.0

# END
